# 4. 작사가 인공지능 만들기
## 1. 데이터읽어오기

In [1]:
import glob
import os, re
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
raw_corpus = []
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you', 'By my side But then I spent so many nights', "Just thinking how you've done me wrong", 'I grew strong', "I learned how to get along And so you're back", 'From outer space', 'I just walked in to find you', 'Here without that look upon your face I should have changed that fucking lock']


## 2. 데이터 정제
### 2-1. 정규표현식을 이용한 corpus생성

In [2]:
# 정규표현식으로 문장 정제하는 함수 
def preprocess_sentence(sentence):
    #     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    #     2. 특수문자 양쪽에 공백을 넣고
    #     3. 여러개의 공백은 하나의 공백으로 바꿉니다
    #     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    #     5. 다시 양쪽 공백을 지웁니다
    #     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [3]:
# corpus에 정제된 문장 모음.
corpus = []
length = 15
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(list(preprocessed_sentence.split()))> length: 
        continue
    else: 
        corpus.append(preprocessed_sentence)
# 정제된 결과를 10개만 확인
print(corpus[:10])
print("총문장수:", len(corpus))

['<start> at first i was afraid <end>', '<start> i was petrified <end>', '<start> i kept thinking i could never live without you <end>', '<start> by my side but then i spent so many nights <end>', '<start> just thinking how you ve done me wrong <end>', '<start> i grew strong <end>', '<start> i learned how to get along and so you re back <end>', '<start> from outer space <end>', '<start> i just walked in to find you <end>', '<start> i would have made you leave your key <end>']
총문장수: 156013


### 2-2. Tokenizer 패키지로 corpus를 텐서로 변환

In [4]:
# tokenize() 함수로 데이터를 Tensor로 변환.
def tokenize(corpus):
    # 텐서플로우에서 제공패키지
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 전체 단어의 개수
        filters=' ',     # 별도로 전처리 로직을 추가할 수 있음. 
        oov_token="<unk>"# out-of-vocabulary
    )
    tokenizer.fit_on_texts(corpus) # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tensor = tokenizer.texts_to_sequences(corpus) # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다(최대길이maxlen 설정가능)
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   70  248 ...    0    0    0]
 [   2    4   53 ...    0    0    0]
 [   2    4 1077 ...    0    0    0]
 ...
 [   2    8    4 ...    0    0    0]
 [   2   44   17 ...    0    0    0]
 [   2    6  172 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f49932f9d10>


In [5]:
# 변환된 텐서 확인
print(tensor[:3])
tensor.shape

[[   2   70  248    4   53  708    3    0    0    0    0    0    0    0
     0]
 [   2    4   53 6263    3    0    0    0    0    0    0    0    0    0
     0]
 [   2    4 1077  531    4  104   80  192  257    7    3    0    0    0
     0]]


(156013, 15)

In [6]:
# tokenizer에 구축된 단어 사전의 인덱스
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
# 소스문장/타겟문장을 만든다. 
src_input = tensor[:, :-1]  # 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:] # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
print(src_input[0]) #소스문장 예시 
print(tgt_input[0]) #타겟문장 예시
len(src_input) #소스문장수

[  2  70 248   4  53 708   3   0   0   0   0   0   0   0]
[ 70 248   4  53 708   3   0   0   0   0   0   0   0   0]


156013

## 3. 평가 데이터셋 분리 

In [8]:
# train,validation set분리 
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.20, random_state=42)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [9]:
# 파라미터 설정
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE


# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset 

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 4. 모델 설계 및 훈련

In [10]:
# TextGenerator클래스 
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()    
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)   
        return out

#세팅
embedding_size = 526
hidden_size = 1024 # 1024
VOCAB_SIZE = tokenizer.num_words + 1   
print(VOCAB_SIZE)
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 실행
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

12001
Epoch 1/10
487/487 [==============================] - 159s 316ms/step - loss: 4.0004
Epoch 2/10
487/487 [==============================] - 156s 319ms/step - loss: 3.0194
Epoch 3/10
487/487 [==============================] - 155s 319ms/step - loss: 2.8365
Epoch 4/10
487/487 [==============================] - 155s 319ms/step - loss: 2.6955
Epoch 5/10
487/487 [==============================] - 155s 318ms/step - loss: 2.5799
Epoch 6/10
487/487 [==============================] - 155s 318ms/step - loss: 2.4628
Epoch 7/10
487/487 [==============================] - 155s 318ms/step - loss: 2.3665
Epoch 8/10
487/487 [==============================] - 155s 317ms/step - loss: 2.2736
Epoch 9/10
487/487 [==============================] - 155s 318ms/step - loss: 2.1840
Epoch 10/10
487/487 [==============================] - 155s 318ms/step - loss: 2.0967


In [11]:
# 문장 생성 함수 
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    return generated



In [12]:
# 문장 생성 함수 실행하여 모델에게 작문 시켜보기 
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m a liar <end> '

In [14]:
generate_text(model, tokenizer, init_sentence="<start> I believe", max_len=20)

'<start> i believe in god , <end> '

## 프로젝트 정리
- text를 컴퓨터가 알아들을 수 있는 형식으로 변경하는 과정(text->숫자)과 단어 예측하는 일련의 과정을 경험해보니 많이 배울 수 있었음. 
- 정규표현식이 복잡하긴 했지만, 사람이 일일이 수고를 하는 것에 비하면 편리한 방법이므로 잘 배워놔야겠다고 생각함. 
- 자원의 효율성을 고려하여 문장 길이를 15단어로 제한하여 학습을 진행했고, 손실지표는 지속적으로 줄어드는 모습을 보였음. 
- 모델에 다양한 문장을 적용하여 예측을 하였는데, 사람이 쓸 법한 결과를 도출하여 학습이 잘된 것 같았음. 
